Needed imports and utility

In [ ]:
import os
import ansys.speos.script as script
import ansys.speos.core as core

tests_data_path = os.path.join(os.path.abspath(''), os.path.pardir, "assets")

def clean_all_dbs(speos_client: core.SpeosClient):
    for item in (
        speos_client.jobs().list()
        + speos_client.scenes().list()
        + speos_client.simulation_templates().list()
        + speos_client.sensor_templates().list()
        + speos_client.source_templates().list()
        + speos_client.intensity_templates().list()
        + speos_client.spectrums().list()
        + speos_client.vop_templates().list()
        + speos_client.sop_templates().list()
        + speos_client.parts().list()
        + speos_client.bodies().list()
        + speos_client.faces().list()
    ):
        item.delete()

Create connection with speos rpc server

In [ ]:
speos = core.Speos(host="localhost", port=50098)
clean_all_dbs(speos.client)

New project

In [ ]:
p = script.Project(speos=speos)
print(p)

# Create
Create locally  
The mention "local: " is added when printing the sensor

In [ ]:
distortion_file_path = os.path.join(tests_data_path, "CameraInputFiles", "CameraDistortion_130deg.OPTDistortion")
transmittance_file_path = os.path.join(tests_data_path, "CameraInputFiles", "CameraTransmittance.spectrum")
blue_spectrum_path = os.path.join(tests_data_path, "CameraInputFiles", "CameraSensitivityBlue.spectrum")
green_spectrum_path = os.path.join(tests_data_path, "CameraInputFiles", "CameraSensitivityGreen.spectrum")
red_spectrum_path = os.path.join(tests_data_path, "CameraInputFiles", "CameraSensitivityRed.spectrum")

sensor1 = p.create_sensor(name="Sensor.1", feature_type=script.sensor.Camera)
sensor1.set_distortion_file_uri(uri=distortion_file_path).set_mode_photometric().set_transmittance_file_uri(uri=transmittance_file_path)\
                .set_mode_color().set_blue_spectrum_file_uri(uri=blue_spectrum_path).set_green_spectrum_file_uri(uri=green_spectrum_path)\
                .set_red_spectrum_file_uri(uri=red_spectrum_path)
print(sensor1)

Push it to the server.  
Now that it is committed to the server, the mention "local: " is no more present when printing the sensor.  

In [ ]:
sensor1.commit()
print(sensor1)

Another example by setting more characteristics

In [ ]:
distortion_file_path = os.path.join(tests_data_path, "CameraInputFiles", "CameraDistortion_130deg.OPTDistortion")
transmittance_file_path = os.path.join(tests_data_path, "CameraInputFiles", "CameraTransmittance.spectrum")
blue_spectrum_path = os.path.join(tests_data_path, "CameraInputFiles", "CameraSensitivityBlue.spectrum")
green_spectrum_path = os.path.join(tests_data_path, "CameraInputFiles", "CameraSensitivityGreen.spectrum")
red_spectrum_path = os.path.join(tests_data_path, "CameraInputFiles", "CameraSensitivityRed.spectrum")

sensor2 = p.create_sensor(name="Sensor.2", feature_type=script.sensor.Camera)
sensor2.set_distortion_file_uri(uri=distortion_file_path)\
        .set_mode_photometric().set_transmittance_file_uri(uri=transmittance_file_path)\
                                .set_layer_type_source()\
                                .set_mode_color().set_blue_spectrum_file_uri(uri=blue_spectrum_path)\
                                                .set_green_spectrum_file_uri(uri=green_spectrum_path)\
                                                .set_red_spectrum_file_uri(uri=red_spectrum_path)
sensor2.set_focal_length(5.5)\
        .set_height(value=6)\
        .set_width(value=6)\
        .set_axis_system([20,10,40,1,0,0,0,1,0,0,0,1])
sensor2.commit()
print(sensor2)

## Default values
Some default values are available when applicable in every methods and class.

# Read
## Sensor Instance
A mention "local: " is added if it is not yet committed to the server

In [ ]:
print(sensor1)

## Project

In [ ]:
print(p)

# Update
Tipp: if you are manipulating a source already committed, don't forget to commit your changes.  
If you don't, you will still only watch what is committed on the server.

In [ ]:
sensor1.set_f_number(value=11)\
        .set_axis_system([17, 10, 10, 1, 0, 0, 0, 1, 0, 0, 0, 1])
sensor1.commit()
print(sensor1)

## Reset
Possibility to reset local values from the one available in the server.

In [ ]:
sensor1.set_mode_geometric() # set camera in geometric mode but no commit
sensor1.reset() # reset -> this will apply the server value to the local value
sensor1.delete() # delete (to display the local value with the below print)
print(sensor1._sensor_template)

# Delete
Once the data is deleted from the server, you can still work with local data and maybe commit later.

In [ ]:
sensor2.delete()
print(sensor2)

In [ ]:
sensor1.delete()

# More content
## Irradiance sensor

In [ ]:
sensor3 = p.create_sensor(name="Sensor.3")
sensor3.commit()
print(sensor3)

In [ ]:
sensor3.set_type_colorimetric()
sensor3.set_layer_type_polarization()
sensor3.commit()
print(sensor3)

In [ ]:
sensor3.delete()

## radiance sensor

In [ ]:
sensor4 = p.create_sensor(name="Sensor.4", feature_type=script.Radiance)
sensor4.commit()
print(sensor4)

In [ ]:
sensor4.set_focal(value=200).set_type_spectral()
sensor4.set_layer_type_source()
sensor4.commit()
print(sensor4)

In [ ]:
sensor4.delete()
print(sensor4)

# Cleanup

In [ ]:
clean_all_dbs(speos.client)